# Import Libraries
#### Standard

In [1]:
import os
import numpy as np
import pandas as pd

#### Visualization

In [2]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import colors
from matplotlib import rc
import seaborn as sns
import networkx as nx

# Import Data
Data Preprocessing 단계에서 생성된 통합 데이터 테이블을 불러온다

In [3]:
root = os.path.join(os.getcwd(), 'DATA')

df_raw = pd.read_csv(os.path.join(root, 'raw_data.csv'), index_col = 0)
df_project = pd.read_csv(os.path.join(root, 'project_data.csv'), index_col = 0)

code_jeom = pd.read_csv(os.path.join(root, 'jeom_code.csv'), index_col = 0)
code_jikwhi = pd.read_csv(os.path.join(root, 'jikwhi_code.csv'), index_col = 0)
code_jkmu = pd.read_csv(os.path.join(root, 'jkmu_code.csv'), index_col = 0)

직원 데이터는 지속적으로 참고해야 한다

In [4]:
df_jikwon = pd.read_csv(os.path.join(root, 'jikwon.csv'), encoding='cp949')

----

# Drop Data


요청구분은 무수정, 수정, 신규, 폐기, 총 4종류가 있는데 우리는 신규/수정한 이력만을 볼 것이다  
이를 통해 프로그램을 신규/수정한 이력이 없는 직원들도 제거된다

In [5]:
df_raw[['요청구분', '요청번호']].groupby('요청구분').count()

,요청번호
요청구분,
무수정,5557
수정,189113
신규,166315
폐기,8934


In [6]:
print(df_raw.shape, end="->")
df = df_raw[(df_raw['요청구분'] == '신규') | (df_raw['요청구분'] == '수정')]
print(df.shape)

(370040, 23)->(355428, 23)


In [7]:
df['요청구분'].unique()

array(['수정', '신규'], dtype=object)

# 분류하기

### 프로그램종류
프로그램 종류를 파악해서 어떤 언어/툴 등을 사용했는지 파악 가능

In [8]:
df[['프로그램종류']].drop_duplicates().reset_index(drop=True)

,프로그램종류
0,온라인서비스
1,채널화면_txt
2,배치
3,거래코드
4,채널화면_MAP
5,IO포맷 ALL
6,DBIO MAP
7,DBIO
8,공통모듈
9,채널화면_mrd


기술 스택을 파악하기에는 프로그램 종류에 대한 정보가 조금은 부족한 것 같다

각 프로그램 종류에 따라 어떤 언어나 툴을 사용하는지 파악할 수 있다면 좀 더 다양성을 줄이고 포괄적으로 데이터를 볼 수 있을 듯 하다

언어/툴/종류에 따라..

**몇가지 알아낸것**
1. 신자금세탁방지(AP) 프로그램종류 BAT_ : 배치성 코드
2. 신자금세탁방지(AP) 프로그램종류 ONL_ : 서비스 코드
3. 금융개발부 DBIO / DBIO MAP : 프로프레임3.0 // 글로개발부 DBI / DBIO MAP : 프로프레임4.0
>  금융개발부는 프로프레임3.0 사용 <-> 글로벌개발부은 프로프레임4.0 사용
4. 금융개발부 IO포맷 / 화면매핑 : iDev@BuildLink
5. 금융개발부 화면 : iDev@Tool
6. ICT운영부 클라우드 : 내부 gitlab 를 사용하여 프로젝트 관리 > 형상관리에서 끌어다가 GIT프로젝트로 표현


In [9]:
def add_class(row):
    program = row['프로그램종류']
    if program == '.net(dll)':
        return "VB"
    elif program == '2PC배치':
        return "C"
    elif program == 'BAT_Bean':
        return "JAVA"
    elif program == 'BAT_DBIO':
        return "JAVA"
    elif program == 'BAT_JAVA':
        return "JAVA"
    elif program == 'BAT_OMM':
        return "JAVA"
    elif program == 'BAT_Shell':
        return "JAVA"
    elif program == 'BAT_배치':
        return "JAVA"
    elif program == 'CONFIG':
        return "Shell"
    elif program == 'CTL':
        return "JAVA"
    elif program == 'DBIO':
        return "프로프레임"
    elif program == 'DBIO MAP':
        return "프로프레임"
    elif program == 'FLEX(.mxml)':
        return "FLEX"        
    elif program == 'FLEX(.swf)':
        return "FLEX"        
    elif program == 'FLEX(/bin 파일)':
        return "FLEX"
    elif program == 'FLEX(/src 파일)':
        return "FLEX"
    elif program == 'GIT프로젝트':
        return "Cloud"
    elif program == 'HTML':
        return "HTML "
    elif program == 'HTML5_html':
        return "HTML "
    elif program == 'HTML5_xml':
        return "HTML "
    elif program == 'HTML5_스크립트':
        return "HTML "
    elif program == 'HTML5_이미지':
        return "HTML "
    elif program == 'Header FILE':
        return "HEADER"
    elif program == 'IO포맷':
        return "iDev@BuildLink"
    elif program == 'IO포맷 ALL':
        return "iDev@BuildLink"
    elif program == 'Image':
        return "기타"
    elif program == 'Java':
        return "JAVA"
    elif program == 'MODULE':
        return "C"
    elif program == 'ONL_Bean':
        return "JAVA"
    elif program == 'ONL_DBIO':
        return "JAVA"
    elif program == 'ONL_JAVA':
        return "JAVA"
    elif program == 'ONL_JAVA Module':
        return "JAVA"
    elif program == 'ONL_OMM':
        return "JAVA"
    elif program == 'PACKAGE':
        return "기타"
    elif program == 'SCRIPT':
        return "Shell"
    elif program == 'SHELL':
        return "Shell"
    elif program == 'SQL':
        return "SQL"
    elif program == 'XML':
        return "xml"
    elif program == '거래코드':
        return "C"
    elif program == '공통모듈':
        return "C"
    elif program == '배치':
        return "C"        
    elif program == '배치(*.pc)':
        return "C"
    elif program == '배치코드':
        return "C"
    elif program == '상주배치':
        return "C"
    elif program == '스크립트(긴급)':
        return "Shell"
    elif program == '스크립트(일반)':
        return "Shell"
    elif program == '실행모듈(서버)':
        return "Server"
    elif program == '연동코드':
        return "C"
    elif program == '온라인서비스':
        return "C"
    elif program.startswith('웹'):
        return "Web"
    elif program == '채널매핑룰':
        return "iDev@BuildLink"
    elif program == '채널매핑룰.chl':
        return "iDev@BuildLink"
    elif program == '채널매핑룰.rule':
        return "iDev@BuildLink"
    elif program == '채널화면_MAP':
        return "iDev@Tool"
    elif program == '채널화면_js':
        return "iDev@Tool"
    elif program == '채널화면_mrd':
        return "iDev@Tool"
    elif program == '채널화면_scn':
        return "iDev@Tool"
    elif program == '채널화면_trn':
        return "iDev@Tool"
    elif program == '채널화면_txt':
        return "iDev@Tool"
    elif program == '프로프레임소스(.c)':
        return "C"
    elif program == '화면_VB':
        return "VB"
    elif program == '후행성배치':
        return "C"

In [10]:
df['A'] = df.apply(lambda x: add_class(x), axis=1)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
df['A_COUNT'] = df.groupby(['A', '프로그램종류'])[['JIKWON_NO']].transform('count')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
df['B_COUNT'] = df.groupby(['시스템유형', '업무그룹', '업무'])[['JIKWON_NO']].transform('count')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
df.head(2)

,JIKWON_NO,JEOM_NO,JIKGUN,JIKGEUB,JIKWHI,JIKWHI2,JUJKMU_C,JUJKMU_RATE,BUJKMU_C,BUJKMU_RATE,...,프로그램종류,프로그램명,프로젝트번호,요청일자,요청구분,요청번호,프로그램경로,A,A_COUNT,B_COUNT
0,20101491,63,20004,5,20164,3,11609.0,100.0,NaN,0.0,...,온라인서비스,sfis2100d.c,P2021-00342-01,2021-01-20,수정,2021-04-002421,/nbsdev/fib/fis/src,C,15617,531
1,20101491,63,20004,5,20164,3,11609.0,100.0,NaN,0.0,...,온라인서비스,sfis7100a.c,P2021-00342-01,2021-01-20,수정,2021-04-002421,/nbsdev/fib/fis/src,C,15617,531


# Export Data

In [14]:
data_for_dash2 = pd.merge(left=df[['JIKWON_NO', '시스템유형', '업무그룹', '업무', '프로그램종류', '프로그램명', 'A', 'A_COUNT', 'B_COUNT']],
                         right=df_jikwon[['JIKWON_NO', 'NAME']],
                         how='left', left_on='JIKWON_NO', right_on='JIKWON_NO')

In [15]:
data_for_dash2

,JIKWON_NO,시스템유형,업무그룹,업무,프로그램종류,프로그램명,A,A_COUNT,B_COUNT,NAME
0,20101491,코어,External Channel,펌뱅킹,온라인서비스,sfis2100d.c,C,15617,531,신범철
1,20101491,코어,External Channel,펌뱅킹,온라인서비스,sfis7100a.c,C,15617,531,신범철
2,20101491,코어,External Channel,펌뱅킹,채널화면_txt,5639000000.txt,iDev@Tool,5246,531,신범철
3,20101491,코어,External Interface,가상계좌,온라인서비스,svas1102c.c,C,15617,72,신범철
4,20101491,코어,단위I/F공통,전자금융,온라인서비스,selb0002a.c,C,15617,160,신범철
5,20101491,코어,External Interface,가상계좌,채널화면_txt,6004230000.txt,iDev@Tool,5246,72,신범철
6,20101491,코어,External Interface,가상계좌,배치,bvas_vrfy_divrst.c,C,5398,72,신범철
7,20101491,코어,External Channel,펌뱅킹,채널화면_txt,5631050000.txt,iDev@Tool,5246,531,신범철
8,20101491,코어,External Interface,가상계좌,채널화면_txt,6006170000.txt,iDev@Tool,5246,72,신범철
9,20101491,코어,External Interface,가상계좌,채널화면_txt,6004060000.txt,iDev@Tool,5246,72,신범철


In [16]:
data_for_dash2.to_csv(os.path.join(root, 'data_for_dash2.csv'), na_rep='NaN')